# Detección de Fraude en Transacciones con Tarjetas de Crédito
## Metodología CRISP-DM

**Dataset:** Credit Card Fraud Detection (Kaggle)

---

## 1. Comprensión del Negocio

### Problema
El fraude con tarjetas de crédito representa pérdidas millonarias para instituciones financieras y afecta la confianza de los clientes. Es crucial identificar transacciones fraudulentas de manera automática y en tiempo real.

### Objetivo
Desarrollar un modelo de machine learning que permita identificar transacciones fraudulentas con alta precisión, minimizando falsos positivos (transacciones legítimas marcadas como fraude) y falsos negativos (fraudes no detectados).

### Contexto del Dataset
- Contiene transacciones realizadas con tarjetas de crédito en septiembre de 2013 por usuarios europeos
- Las transacciones ocurrieron en un período de 2 días
- Dataset altamente desbalanceado: solo 0.172% de las transacciones son fraudulentas
- Por razones de confidencialidad, las características originales han sido transformadas usando PCA

### Métricas de Éxito
- **Recall (Sensibilidad):** Capacidad de detectar fraudes reales
- **Precision:** Evitar falsos positivos que afecten la experiencia del cliente
- **F1-Score:** Balance entre precision y recall
- **AUC-ROC:** Capacidad general de discriminación del modelo

In [ ]:
# Instalación de librerías necesarias
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, precision_recall_curve,
    f1_score, accuracy_score, precision_score, recall_score
)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Librerías importadas exitosamente")

---
## 2. Comprensión de los Datos

En esta fase exploraremos y entenderemos la estructura, calidad y características del dataset.

In [ ]:
# Carga del dataset
# NOTA: Descarga el dataset de Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud
# y colócalo en la ruta apropiada

df = pd.read_csv('creditcard.csv')

print("Dataset cargado exitosamente")
print(f"Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")

In [ ]:
# Primeras filas del dataset
df.head(10)

In [ ]:
# Información general del dataset
print("=" * 60)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 60)
df.describe()

In [ ]:
# Análisis de valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())
print(f"\nTotal de valores nulos: {df.isnull().sum().sum()}")

In [ ]:
# Análisis del desbalance de clases
print("=" * 60)
print("ANÁLISIS DE LA VARIABLE OBJETIVO (Class)")
print("=" * 60)

class_distribution = df['Class'].value_counts()
print("\nConteo de clases:")
print(class_distribution)

print("\nProporción de clases:")
print(df['Class'].value_counts(normalize=True) * 100)

fraud_percentage = (df['Class'].sum() / len(df)) * 100
print(f"\nPorcentaje de fraudes: {fraud_percentage:.3f}%")
print(f"Ratio de desbalance: 1:{int(class_distribution[0]/class_distribution[1])}")

In [ ]:
# Visualización del desbalance de clases
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
class_counts = df['Class'].value_counts()
axes[0].bar(['Normal', 'Fraude'], class_counts.values, color=['#2ecc71', '#e74c3c'])
axes[0].set_ylabel('Cantidad de Transacciones', fontsize=12)
axes[0].set_title('Distribución de Clases', fontsize=14, fontweight='bold')
axes[0].set_yscale('log')
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v, f'{v:,}', ha='center', va='bottom', fontsize=11)

# Gráfico de pastel
colors = ['#2ecc71', '#e74c3c']
explode = (0, 0.1)
axes[1].pie(class_counts.values, labels=['Normal', 'Fraude'], autopct='%1.3f%%',
            colors=colors, explode=explode, shadow=True, startangle=90)
axes[1].set_title('Proporción de Clases', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de las variables Time y Amount
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Distribución de Time
axes[0, 0].hist(df['Time'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Tiempo (segundos)', fontsize=11)
axes[0, 0].set_ylabel('Frecuencia', fontsize=11)
axes[0, 0].set_title('Distribución de Time', fontsize=13, fontweight='bold')

# Distribución de Amount
axes[0, 1].hist(df['Amount'], bins=50, color='lightcoral', edgecolor='black')
axes[0, 1].set_xlabel('Monto ($)', fontsize=11)
axes[0, 1].set_ylabel('Frecuencia', fontsize=11)
axes[0, 1].set_title('Distribución de Amount', fontsize=13, fontweight='bold')
axes[0, 1].set_xlim([0, 1000])

# Time por clase
df[df['Class'] == 0]['Time'].hist(bins=50, alpha=0.6, label='Normal', ax=axes[1, 0], color='green')
df[df['Class'] == 1]['Time'].hist(bins=50, alpha=0.6, label='Fraude', ax=axes[1, 0], color='red')
axes[1, 0].set_xlabel('Tiempo (segundos)', fontsize=11)
axes[1, 0].set_ylabel('Frecuencia', fontsize=11)
axes[1, 0].set_title('Distribución de Time por Clase', fontsize=13, fontweight='bold')
axes[1, 0].legend()

# Amount por clase
df[df['Class'] == 0]['Amount'].hist(bins=50, alpha=0.6, label='Normal', ax=axes[1, 1], color='green')
df[df['Class'] == 1]['Amount'].hist(bins=50, alpha=0.6, label='Fraude', ax=axes[1, 1], color='red')
axes[1, 1].set_xlabel('Monto ($)', fontsize=11)
axes[1, 1].set_ylabel('Frecuencia', fontsize=11)
axes[1, 1].set_title('Distribución de Amount por Clase', fontsize=13, fontweight='bold')
axes[1, 1].set_xlim([0, 500])
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Estadísticas de Amount por clase
print("Estadísticas de Amount por clase:")
print("\nTransacciones Normales:")
print(df[df['Class'] == 0]['Amount'].describe())
print("\nTransacciones Fraudulentas:")
print(df[df['Class'] == 1]['Amount'].describe())

In [ ]:
# Matriz de correlación de variables principales
# Seleccionamos algunas variables V para visualizar
features_to_plot = ['Time', 'Amount', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'Class']

plt.figure(figsize=(12, 10))
correlation_matrix = df[features_to_plot].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación (Variables Seleccionadas)', fontsize=15, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Correlación de todas las variables con Class
correlations_with_class = df.corr()['Class'].sort_values(ascending=False)
print("Correlación de variables con Class (10 más altas):")
print(correlations_with_class.head(11))  # 11 porque incluye Class consigo misma
print("\nCorrelación de variables con Class (10 más bajas):")
print(correlations_with_class.tail(10))

In [ ]:
# Visualización de las correlaciones más fuertes
plt.figure(figsize=(10, 8))
top_correlations = correlations_with_class[1:16]  # Top 15 (excluyendo Class)
colors_corr = ['red' if x < 0 else 'green' for x in top_correlations.values]
top_correlations.plot(kind='barh', color=colors_corr)
plt.xlabel('Correlación con Class', fontsize=12)
plt.title('Top 15 Variables con Mayor Correlación (Absoluta) con Class', 
          fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

### Conclusiones de la Comprensión de Datos

1. **Dataset limpio:** No hay valores nulos ni duplicados
2. **Desbalance severo:** Solo 0.172% de transacciones son fraudulentas (492 de 284,807)
3. **Variables transformadas:** V1-V28 son componentes principales (PCA), por lo que no tienen interpretación directa
4. **Características originales:** Time y Amount son las únicas variables no transformadas
5. **Patrones detectados:** 
   - Los fraudes tienden a tener montos menores en promedio
   - Algunas variables V muestran correlación significativa con fraude (V14, V4, V11, V17)
6. **Desafío principal:** El desbalance extremo requerirá técnicas especiales de balanceo

---
## 3. Preparación de los Datos

En esta fase limpiaremos y transformaremos los datos para que sean adecuados para el modelado.

In [ ]:
# Crear copia del dataset para trabajar
df_processed = df.copy()

print("Dataset copiado para procesamiento")
print(f"Shape: {df_processed.shape}")

In [ ]:
# Normalización de Time y Amount
# Usamos RobustScaler porque es menos sensible a outliers

scaler = RobustScaler()

df_processed['Time_scaled'] = scaler.fit_transform(df_processed['Time'].values.reshape(-1, 1))
df_processed['Amount_scaled'] = scaler.fit_transform(df_processed['Amount'].values.reshape(-1, 1))

# Eliminar las columnas originales
df_processed.drop(['Time', 'Amount'], axis=1, inplace=True)

print("✓ Variables Time y Amount escaladas")
print(f"Shape actual: {df_processed.shape}")

In [ ]:
# Verificar el resultado del escalado
print("Estadísticas después del escalado:")
print(df_processed[['Time_scaled', 'Amount_scaled']].describe())

In [ ]:
# Separar características (X) y variable objetivo (y)
X = df_processed.drop('Class', axis=1)
y = df_processed['Class']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nDistribución de y:")
print(y.value_counts())

In [ ]:
# División del dataset en entrenamiento y prueba (80-20)
# Usamos stratify para mantener la proporción de clases

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("División del dataset:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

print("\nDistribución en conjunto de entrenamiento:")
print(y_train.value_counts())
print(y_train.value_counts(normalize=True) * 100)

print("\nDistribución en conjunto de prueba:")
print(y_test.value_counts())
print(y_test.value_counts(normalize=True) * 100)

### Técnicas de Balanceo de Clases

Debido al desbalance extremo, implementaremos tres estrategias:
1. **Sin balanceo** (baseline)
2. **SMOTE** (Synthetic Minority Over-sampling Technique)
3. **Random Under-sampling**

In [ ]:
# Estrategia 1: Dataset sin balanceo (baseline)
X_train_original = X_train.copy()
y_train_original = y_train.copy()

print("Dataset original (sin balanceo):")
print(f"X_train shape: {X_train_original.shape}")
print(f"Distribución: {y_train_original.value_counts().to_dict()}")

In [ ]:
# Estrategia 2: SMOTE (Oversampling de la clase minoritaria)
smote = SMOTE(random_state=42, sampling_strategy=0.5)  # Balanceo parcial
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Dataset con SMOTE:")
print(f"X_train_smote shape: {X_train_smote.shape}")
print(f"Distribución: {pd.Series(y_train_smote).value_counts().to_dict()}")
print(f"Proporción: {pd.Series(y_train_smote).value_counts(normalize=True).to_dict()}")

In [ ]:
# Estrategia 3: Random Under-sampling (Reducir la clase mayoritaria)
rus = RandomUnderSampler(random_state=42, sampling_strategy=0.5)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

print("Dataset con Random Under-sampling:")
print(f"X_train_rus shape: {X_train_rus.shape}")
print(f"Distribución: {pd.Series(y_train_rus).value_counts().to_dict()}")
print(f"Proporción: {pd.Series(y_train_rus).value_counts(normalize=True).to_dict()}")

In [ ]:
# Visualización de las tres estrategias
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

strategies = [
    (y_train_original, 'Sin Balanceo'),
    (y_train_smote, 'Con SMOTE'),
    (y_train_rus, 'Con Under-sampling')
]

for idx, (y_data, title) in enumerate(strategies):
    counts = pd.Series(y_data).value_counts().sort_index()
    axes[idx].bar(['Normal', 'Fraude'], counts.values, color=['#2ecc71', '#e74c3c'])
    axes[idx].set_ylabel('Cantidad', fontsize=11)
    axes[idx].set_title(title, fontsize=13, fontweight='bold')
    for i, v in enumerate(counts.values):
        axes[idx].text(i, v, f'{v:,}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

### Conclusiones de la Preparación de Datos

1. **Escalado:** Variables Time y Amount normalizadas usando RobustScaler
2. **División:** 80% entrenamiento, 20% prueba con estratificación
3. **Estrategias de balanceo:**
   - Original: 227,451 normales / 394 fraudes
   - SMOTE: 227,451 normales / 113,725 fraudes (sintéticos)
   - Under-sampling: 788 normales / 394 fraudes
4. **Próximo paso:** Entrenar modelos con cada estrategia y comparar resultados

---
## 4. Modelado

Entrenaremos múltiples algoritmos de clasificación con diferentes estrategias de balanceo.

### 4.1 Definición de Modelos

In [ ]:
# Definir los modelos a entrenar
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100, max_depth=15),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100, max_depth=5)
}

print("Modelos definidos:")
for name in models.keys():
    print(f"  - {name}")

### 4.2 Entrenamiento con Dataset Original (Sin Balanceo)

In [ ]:
# Diccionario para almacenar modelos entrenados
trained_models_original = {}

print("="*60)
print("ENTRENAMIENTO CON DATASET ORIGINAL (SIN BALANCEO)")
print("="*60)

for name, model in models.items():
    print(f"\n Entrenando {name}...")
    model.fit(X_train_original, y_train_original)
    trained_models_original[name] = model
    print(f"✓ {name} entrenado")

print("\n✓ Todos los modelos entrenados con dataset original")

### 4.3 Entrenamiento con SMOTE

In [ ]:
# Diccionario para almacenar modelos entrenados con SMOTE
trained_models_smote = {}

print("="*60)
print("ENTRENAMIENTO CON SMOTE")
print("="*60)

for name, model_class in models.items():
    print(f"\n Entrenando {name}...")
    # Crear nueva instancia del modelo
    if name == 'Logistic Regression':
        model = LogisticRegression(random_state=42, max_iter=1000)
    elif name == 'Decision Tree':
        model = DecisionTreeClassifier(random_state=42, max_depth=10)
    elif name == 'Random Forest':
        model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=15)
    else:  # Gradient Boosting
        model = GradientBoostingClassifier(random_state=42, n_estimators=100, max_depth=5)
    
    model.fit(X_train_smote, y_train_smote)
    trained_models_smote[name] = model
    print(f"✓ {name} entrenado")

print("\n✓ Todos los modelos entrenados con SMOTE")

### 4.4 Entrenamiento con Under-sampling

In [ ]:
# Diccionario para almacenar modelos entrenados con Under-sampling
trained_models_rus = {}

print("="*60)
print("ENTRENAMIENTO CON RANDOM UNDER-SAMPLING")
print("="*60)

for name, model_class in models.items():
    print(f"\n Entrenando {name}...")
    # Crear nueva instancia del modelo
    if name == 'Logistic Regression':
        model = LogisticRegression(random_state=42, max_iter=1000)
    elif name == 'Decision Tree':
        model = DecisionTreeClassifier(random_state=42, max_depth=10)
    elif name == 'Random Forest':
        model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=15)
    else:  # Gradient Boosting
        model = GradientBoostingClassifier(random_state=42, n_estimators=100, max_depth=5)
    
    model.fit(X_train_rus, y_train_rus)
    trained_models_rus[name] = model
    print(f"✓ {name} entrenado")

print("\n✓ Todos los modelos entrenados con Under-sampling")

### Resumen del Modelado

Se han entrenado 12 modelos en total:
- 4 algoritmos diferentes
- 3 estrategias de balanceo
- Listos para evaluación en la siguiente fase

---
## 5. Evaluación

Evaluaremos el rendimiento de todos los modelos usando métricas apropiadas para problemas de clasificación desbalanceada.

In [ ]:
# Función para evaluar modelos
def evaluate_model(model, X_test, y_test, model_name, strategy_name):
    """
    Evalúa un modelo y retorna métricas de rendimiento
    """
    # Predicciones
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    results = {
        'Model': model_name,
        'Strategy': strategy_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    return results

print("✓ Función de evaluación definida")

### 5.1 Evaluación de Modelos con Dataset Original

In [ ]:
# Evaluar modelos entrenados con dataset original
results_original = []

print("="*60)
print("EVALUACIÓN: MODELOS CON DATASET ORIGINAL")
print("="*60)

for name, model in trained_models_original.items():
    print(f"\nEvaluando {name}...")
    result = evaluate_model(model, X_test, y_test, name, 'Original')
    results_original.append(result)
    
    print(f"  Accuracy: {result['Accuracy']:.4f}")
    print(f"  Precision: {result['Precision']:.4f}")
    print(f"  Recall: {result['Recall']:.4f}")
    print(f"  F1-Score: {result['F1-Score']:.4f}")
    print(f"  ROC-AUC: {result['ROC-AUC']:.4f}")

print("\n✓ Evaluación completada")

### 5.2 Evaluación de Modelos con SMOTE

In [ ]:
# Evaluar modelos entrenados con SMOTE
results_smote = []

print("="*60)
print("EVALUACIÓN: MODELOS CON SMOTE")
print("="*60)

for name, model in trained_models_smote.items():
    print(f"\nEvaluando {name}...")
    result = evaluate_model(model, X_test, y_test, name, 'SMOTE')
    results_smote.append(result)
    
    print(f"  Accuracy: {result['Accuracy']:.4f}")
    print(f"  Precision: {result['Precision']:.4f}")
    print(f"  Recall: {result['Recall']:.4f}")
    print(f"  F1-Score: {result['F1-Score']:.4f}")
    print(f"  ROC-AUC: {result['ROC-AUC']:.4f}")

print("\n✓ Evaluación completada")

### 5.3 Evaluación de Modelos con Under-sampling

In [ ]:
# Evaluar modelos entrenados con Under-sampling
results_rus = []

print("="*60)
print("EVALUACIÓN: MODELOS CON UNDER-SAMPLING")
print("="*60)

for name, model in trained_models_rus.items():
    print(f"\nEvaluando {name}...")
    result = evaluate_model(model, X_test, y_test, name, 'Under-sampling')
    results_rus.append(result)
    
    print(f"  Accuracy: {result['Accuracy']:.4f}")
    print(f"  Precision: {result['Precision']:.4f}")
    print(f"  Recall: {result['Recall']:.4f}")
    print(f"  F1-Score: {result['F1-Score']:.4f}")
    print(f"  ROC-AUC: {result['ROC-AUC']:.4f}")

print("\n✓ Evaluación completada")

### 5.4 Comparación de Resultados

In [ ]:
# Consolidar todos los resultados
all_results = results_original + results_smote + results_rus

# Crear DataFrame con resultados
df_results = pd.DataFrame([{
    'Model': r['Model'],
    'Strategy': r['Strategy'],
    'Accuracy': r['Accuracy'],
    'Precision': r['Precision'],
    'Recall': r['Recall'],
    'F1-Score': r['F1-Score'],
    'ROC-AUC': r['ROC-AUC']
} for r in all_results])

# Ordenar por F1-Score
df_results_sorted = df_results.sort_values('F1-Score', ascending=False)

print("="*80)
print("TABLA COMPARATIVA DE RESULTADOS")
print("="*80)
print(df_results_sorted.to_string(index=False))
print("\n")

In [ ]:
# Visualización comparativa de métricas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, metric in enumerate(metrics):
    row = idx // 2
    col = idx % 2
    
    pivot_data = df_results.pivot(index='Model', columns='Strategy', values=metric)
    pivot_data.plot(kind='bar', ax=axes[row, col], width=0.8)
    
    axes[row, col].set_title(f'{metric} por Modelo y Estrategia', 
                            fontsize=13, fontweight='bold')
    axes[row, col].set_ylabel(metric, fontsize=11)
    axes[row, col].set_xlabel('Modelo', fontsize=11)
    axes[row, col].legend(title='Estrategia', loc='best')
    axes[row, col].tick_params(axis='x', rotation=45)
    axes[row, col].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de ROC-AUC comparativo
plt.figure(figsize=(12, 6))
pivot_roc = df_results.pivot(index='Model', columns='Strategy', values='ROC-AUC')
pivot_roc.plot(kind='bar', width=0.8, color=['#3498db', '#e74c3c', '#2ecc71'])
plt.title('Comparación de ROC-AUC por Modelo y Estrategia', fontsize=15, fontweight='bold')
plt.ylabel('ROC-AUC Score', fontsize=12)
plt.xlabel('Modelo', fontsize=12)
plt.legend(title='Estrategia', loc='best')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.5 Matrices de Confusión del Mejor Modelo

In [ ]:
# Identificar el mejor modelo (mayor F1-Score)
best_model_idx = df_results_sorted.index[0]
best_result = all_results[best_model_idx]

print("="*60)
print("MEJOR MODELO")
print("="*60)
print(f"Modelo: {best_result['Model']}")
print(f"Estrategia: {best_result['Strategy']}")
print(f"F1-Score: {best_result['F1-Score']:.4f}")
print(f"ROC-AUC: {best_result['ROC-AUC']:.4f}")
print(f"Recall: {best_result['Recall']:.4f}")
print(f"Precision: {best_result['Precision']:.4f}")

In [ ]:
# Visualizar matrices de confusión para cada estrategia
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

strategies_results = [
    (results_original, 'Original'),
    (results_smote, 'SMOTE'),
    (results_rus, 'Under-sampling')
]

# Buscar el mejor modelo dentro de cada estrategia
for idx, (results_list, strategy_name) in enumerate(strategies_results):
    # Encontrar el mejor modelo de esta estrategia
    best_in_strategy = max(results_list, key=lambda x: x['F1-Score'])
    
    cm = np.array([[best_in_strategy['TN'], best_in_strategy['FP']],
                   [best_in_strategy['FN'], best_in_strategy['TP']]])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Normal', 'Fraude'],
                yticklabels=['Normal', 'Fraude'])
    
    axes[idx].set_title(f'{strategy_name}\n{best_in_strategy["Model"]}\n(F1: {best_in_strategy["F1-Score"]:.4f})',
                       fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Clase Real', fontsize=11)
    axes[idx].set_xlabel('Clase Predicha', fontsize=11)

plt.tight_layout()
plt.show()

### 5.6 Curvas ROC

In [ ]:
# Graficar curvas ROC para los mejores modelos de cada estrategia
plt.figure(figsize=(10, 8))

for results_list, strategy_name in strategies_results:
    best_in_strategy = max(results_list, key=lambda x: x['F1-Score'])
    
    fpr, tpr, _ = roc_curve(y_test, best_in_strategy['probabilities'])
    roc_auc = best_in_strategy['ROC-AUC']
    
    plt.plot(fpr, tpr, linewidth=2,
             label=f'{strategy_name} - {best_in_strategy["Model"]} (AUC = {roc_auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier (AUC = 0.5000)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Curvas ROC - Mejores Modelos por Estrategia', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 5.7 Curvas Precision-Recall

In [ ]:
# Graficar curvas Precision-Recall para los mejores modelos
plt.figure(figsize=(10, 8))

for results_list, strategy_name in strategies_results:
    best_in_strategy = max(results_list, key=lambda x: x['F1-Score'])
    
    precision_curve, recall_curve, _ = precision_recall_curve(
        y_test, best_in_strategy['probabilities']
    )
    
    plt.plot(recall_curve, precision_curve, linewidth=2,
             label=f'{strategy_name} - {best_in_strategy["Model"]} (F1 = {best_in_strategy["F1-Score"]:.4f})')

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Curvas Precision-Recall - Mejores Modelos por Estrategia', 
          fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 5.8 Análisis Detallado del Mejor Modelo

In [ ]:
# Reporte de clasificación del mejor modelo
print("="*60)
print("REPORTE DETALLADO DEL MEJOR MODELO")
print("="*60)
print(f"\nModelo: {best_result['Model']}")
print(f"Estrategia: {best_result['Strategy']}")
print("\n" + "="*60)
print("MATRIZ DE CONFUSIÓN")
print("="*60)
print(f"True Negatives (TN):  {best_result['TN']:,}")
print(f"False Positives (FP): {best_result['FP']:,}")
print(f"False Negatives (FN): {best_result['FN']:,}")
print(f"True Positives (TP):  {best_result['TP']:,}")

print("\n" + "="*60)
print("MÉTRICAS DE RENDIMIENTO")
print("="*60)
print(f"Accuracy:  {best_result['Accuracy']:.4f}")
print(f"Precision: {best_result['Precision']:.4f}")
print(f"Recall:    {best_result['Recall']:.4f}")
print(f"F1-Score:  {best_result['F1-Score']:.4f}")
print(f"ROC-AUC:   {best_result['ROC-AUC']:.4f}")

print("\n" + "="*60)
print("REPORTE DE CLASIFICACIÓN")
print("="*60)
print(classification_report(y_test, best_result['predictions'], 
                          target_names=['Normal', 'Fraude']))

### 5.9 Interpretación de Resultados

In [ ]:
# Análisis de errores del mejor modelo
print("="*60)
print("ANÁLISIS DE ERRORES")
print("="*60)

total_fraudes = best_result['TP'] + best_result['FN']
fraudes_detectados = best_result['TP']
fraudes_no_detectados = best_result['FN']

total_normales = best_result['TN'] + best_result['FP']
falsos_positivos = best_result['FP']

print(f"\nDe {total_fraudes} transacciones fraudulentas:")
print(f"  ✓ Detectadas correctamente: {fraudes_detectados} ({(fraudes_detectados/total_fraudes)*100:.2f}%)")
print(f"  ✗ No detectadas (FN): {fraudes_no_detectados} ({(fraudes_no_detectados/total_fraudes)*100:.2f}%)")

print(f"\nDe {total_normales} transacciones normales:")
print(f"  ✗ Marcadas como fraude (FP): {falsos_positivos} ({(falsos_positivos/total_normales)*100:.2f}%)")

print("\n" + "="*60)
print("IMPACTO DE NEGOCIO")
print("="*60)
print(f"\nPorcentaje de fraudes detectados (Recall): {best_result['Recall']*100:.2f}%")
print(f"Tasa de falsos positivos: {(falsos_positivos/total_normales)*100:.2f}%")
print(f"\nInterpretación:")
print(f"- El modelo detecta {best_result['Recall']*100:.1f}% de los fraudes reales")
print(f"- Solo {(falsos_positivos/total_normales)*100:.2f}% de transacciones legítimas son bloqueadas incorrectamente")
print(f"- De cada 100 alertas de fraude, {best_result['Precision']*100:.1f} son fraudes reales")

---
## Conclusiones y Recomendaciones

### Hallazgos Clave

1. **Desafío del Desbalance:**
   - El dataset presenta un desbalance extremo (99.83% normales vs 0.17% fraudes)
   - Las técnicas de balanceo mejoraron significativamente la detección de fraudes

2. **Comparación de Estrategias:**
   - **Dataset Original:** Alta precisión pero bajo recall (muchos fraudes no detectados)
   - **SMOTE:** Mejor balance entre precision y recall
   - **Under-sampling:** Alto recall pero más falsos positivos

3. **Métricas Importantes:**
   - **Recall (Sensibilidad):** Crítico para detectar fraudes reales
   - **Precision:** Importante para no afectar experiencia del cliente con falsos positivos
   - **F1-Score:** Mejor métrica para evaluar el balance global

### Recomendaciones

1. **Para Producción:**
   - Implementar el modelo con mejor F1-Score
   - Establecer umbrales de probabilidad ajustables según el apetito de riesgo
   - Monitorear continuamente el rendimiento del modelo

2. **Mejoras Futuras:**
   - Probar con otros algoritmos (XGBoost, LightGBM, Neural Networks)
   - Implementar feature engineering adicional
   - Realizar grid search para optimización de hiperparámetros
   - Considerar ensambles de modelos

3. **Consideraciones de Negocio:**
   - Definir el costo de falsos positivos vs falsos negativos
   - Implementar sistema de revisión manual para casos de alta incertidumbre
   - Actualizar el modelo periódicamente con nuevos datos

### Siguientes Pasos

1. **Despliegue:**
   - Validar el modelo con datos más recientes
   - Implementar en un entorno de staging
   - Realizar pruebas A/B antes del despliegue completo

2. **Monitoreo:**
   - Establecer dashboards de métricas en tiempo real
   - Configurar alertas para degradación del modelo
   - Documentar casos de error para mejora continua

---

## Referencias

- Dataset: [Credit Card Fraud Detection - Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud)
- Metodología: CRISP-DM (Cross-Industry Standard Process for Data Mining)
- Librería de balanceo: imbalanced-learn
- Algoritmos: scikit-learn

---

**Autor:** [Tu Nombre]  
**Fecha:** Noviembre 2024  
**Versión:** 1.0